In [1]:
import rpy2.robjects as robjects
read_delim = robjects.r('read.delim')
seq_data = read_delim('sequence.index', header=True, stringsAsFactors=False)

In [2]:
print('dataframe has %d columns and %d rows' % (seq_data.ncol, seq_data.nrow))
print(seq_data.colnames)

dataframe has 26 columns and 50066 rows
 [1] "FASTQ_FILE"          "MD5"                 "RUN_ID"             

 [4] "STUDY_ID"            "STUDY_NAME"          "CENTER_NAME"        

 [7] "SUBMISSION_ID"       "SUBMISSION_DATE"     "SAMPLE_ID"          

[10] "SAMPLE_NAME"         "POPULATION"          "EXPERIMENT_ID"      

[13] "INSTRUMENT_PLATFORM" "INSTRUMENT_MODEL"    "LIBRARY_NAME"       

[16] "RUN_NAME"            "RUN_BLOCK_NAME"      "INSERT_SIZE"        

[19] "LIBRARY_LAYOUT"      "PAIRED_FASTQ"        "WITHDRAWN"          

[22] "WITHDRAWN_DATE"      "COMMENT"             "READ_COUNT"         

[25] "BASE_COUNT"          "ANALYSIS_GROUP"     



In [3]:
#Clean up the data
as_integer = robjects.r('as.integer')
match = robjects.r.match
my_col = match('BASE_COUNT', seq_data.colnames)[0]
print(seq_data[my_col - 1])[:3]
seq_data[my_col - 1] = as_integer(seq_data[my_col - 1])
print(seq_data[my_col - 1][:3])

    [1] "334097928"     "344591352"     "5365584"       "74076840"     

    [5] "74076840"      "337974048"     "279466488"     "385018000"    

    [9] "317111112"     "683415"        "97169580"      "97169580"     

   [13] "2669040"       "228605760"     "228605760"     "423095832"    

   [17] "10313055"      "346176540"     "346176540"     "410491152"    

   [21] "243985248"     "2841048"       "40725108"      "40725108"     

   [25] "432493812"     "289651896"     "326926728"     "364698072"    

   [29] "310783644"     "291921084"     "403453224"     "404856468"    

   [33] "6353100"       "285001065"     "285001065"     "4103325"      

   [37] "274272255"     "274272255"     "21069180"      "437656230"    

   [41] "437656230"     "10029780"      "391141620"     "391141620"    

   [45] "62215020"      "268813332"     "268813332"     "6892740"      

   [49] "320986890"     "320986890"     "33011595"      "329098950"    

   [53] "329098950"     "6357583"       "320298929"

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#Create a variable in the R namespace with the name of the first argument in the assign function.
#This variable contains the content of the dataframe from the python namespace, this separates both objects and they are NOT Aliased
robjects.r.assign('seq.data', seq_data)

In [ ]:
import rpy2.robjects.lib.ggplot2 as ggplot2

In [ ]:
#Plot a histogram with the distribution of center names where sequencing lanes were generated
robjects.r.png('out.png')

In [4]:
from rpy2.robjects.functions import SignatureTranslatedFunction
ggplot2.theme = SignatureTranslatedFunction(ggplot2.theme,
            init_prm_translate={'axis_text_x': 'axis.text.x'})
bar = ggplot2.ggplot(seq_data) + ggplot2.geom_bar() + ggplot2.aes_string(x='CENTER_NAME')  + ggplot2.theme(axis_text_x=ggplot2.element_text(angle=90, hjust=1))
bar.plot()
dev_off = robjects.r('dev.off')
dev_off()

NameError: name 'ggplot2' is not defined

In [5]:
robjects.r('yri_ceu <- seq.data[seq.data$POPULATION %in% c("YRI", "CEU") & seq.data$BASE_COUNT < 2E9 & seq.data$READ_COUNT < 3E7, ]')
robjects.r('yri_ceu$POPULATION <- as.factor(yri_ceu$POPULATION)')
robjects.r('yri_ceu$ANALYSIS_GROUP <- as.factor(yri_ceu$ANALYSIS_GROUP)')

RRuntimeError: Error in eval(expr, envir, enclos) : object 'seq.data' not found


In [ ]:
yri_ceu = robjects.r('yri_ceu')
scatter = ggplot2.ggplot(yri_ceu) + ggplot2.geom_point() + ggplot2.aes_string(x='BASE_COUNT', y='READ_COUNT', shape='factor(POPULATION)', col='factor(ANALYSIS_GROUP)')
scatter.plot()

In [6]:
from Bio import Entrez, SeqIO
Entrez.email = 'nsrchemie@gmail.com'

In [7]:

handle = Entrez.esearch(db='nucleotide', term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism]')
rec_list = Entrez.read(handle)
if rec_list['RetMax'] < rec_list['Count']:
    handle = Entrez.esearch(db='nucleotide',term='CRT[Gene Name] AND "Plasmodium falciparum"[Organism]', retmax=rec_list['Count'])
    rec_list = Entrez.read(handle)

In [8]:
id_list = rec_list['IdList']
hdl = Entrez.efetch(db='nucleotide', id=id_list, rettype='gb')

In [9]:
recs = list(SeqIO.parse(hdl,'gb'))

In [10]:
for rec in recs:
    if rec.name == 'KM288867':
        break

In [11]:
for feature in rec.features:
    if feature.type == 'gene':
        print(feature.qualifiers['gene'])
    elif feature.type == 'exon':
        loc = feature.location
        print(loc.start, loc.end, loc.strand)
    else:
        print('not processed:\n%s' % feature)

not processed:
type: source
location: [0:10000](+)
qualifiers:
    Key: clone, Value: ['PF3D7_0709000']
    Key: db_xref, Value: ['taxon:5833']
    Key: mol_type, Value: ['genomic DNA']
    Key: organism, Value: ['Plasmodium falciparum']

['CRT']
not processed:
type: mRNA
location: join{[2751:3543](+), [3720:3989](+), [4168:4341](+), [4513:4646](+), [4799:4871](+), [4994:5070](+), [5166:5249](+), [5376:5427](+), [5564:5621](+), [5769:5862](+), [6055:6100](+), [6247:6302](+), [6471:7598](+)}
qualifiers:
    Key: gene, Value: ['CRT']
    Key: product, Value: ['chloroquine resistance transporter']
Sub-Features
type: mRNA
location: [2751:3543](+)
qualifiers:

type: mRNA
location: [3720:3989](+)
qualifiers:

type: mRNA
location: [4168:4341](+)
qualifiers:

type: mRNA
location: [4513:4646](+)
qualifiers:

type: mRNA
location: [4799:4871](+)
qualifiers:

type: mRNA
location: [4994:5070](+)
qualifiers:

type: mRNA
location: [5166:5249](+)
qualifiers:

type: mRNA
location: [5376:5427](+)
qualif

In [12]:
for name, value in rec.annotations.items():
    print('%s=%s' % (name, value))

organism=Plasmodium falciparum
source=Plasmodium falciparum (malaria parasite P. falciparum)
accessions=['KM288867']
sequence_version=1
data_file_division=INV
date=12-NOV-2014
taxonomy=['Eukaryota', 'Alveolata', 'Apicomplexa', 'Aconoidasida', 'Haemosporida', 'Plasmodiidae', 'Plasmodium', 'Plasmodium (Laverania)']
keywords=['']
references=[Reference(title='Versatile control of Plasmodium falciparum gene expression with an inducible protein-RNA interaction', ...), Reference(title='Direct Submission', ...)]
gi=706072608


In [13]:
from Bio import Medline
refs = rec.annotations['references']
for ref in refs:
    if ref.pubmed_id != '':
        print(ref.pubmed_id)
        handle = Entrez.efetch(db='pubmed', id=[ref.pubmed_id], rettype='medline', retmode='text')
        records= Medline.parse(handle)
        for med_rec in records:
            print(med_rec)

25370483
{'CRDT': ['2014/11/06 06:00'], 'SO': 'Nat Commun. 2014 Nov 5;5:5329. doi: 10.1038/ncomms6329.', 'VI': '5', 'PHST': ['2014/04/15 [received]', '2014/09/20 [accepted]'], 'PT': ['Journal Article', 'Research Support, N.I.H., Extramural', "Research Support, Non-U.S. Gov't"], 'DEP': '20141105', 'IS': '2041-1723 (Electronic) 2041-1723 (Linking)', 'LR': '20150505', 'EDAT': '2014/11/06 06:00', 'PG': '5329', 'MID': ['NIHMS630149'], 'LA': ['eng'], 'PST': 'epublish', 'FAU': ['Goldfless, Stephen J', 'Wagner, Jeffrey C', 'Niles, Jacquin C'], 'TA': 'Nat Commun', 'DA': '20141105', 'TI': 'Versatile control of Plasmodium falciparum gene expression with an inducible protein-RNA interaction.', 'MHDA': '2016/01/13 06:00', 'PMC': 'PMC4223869', 'DCOM': '20160112', 'SI': ['GENBANK/KM288848', 'GENBANK/KM288849', 'GENBANK/KM288850', 'GENBANK/KM288851', 'GENBANK/KM288852', 'GENBANK/KM288853', 'GENBANK/KM288854', 'GENBANK/KM288855', 'GENBANK/KM288856', 'GENBANK/KM288857', 'GENBANK/KM288858', 'GENBANK/KM28

In [14]:
from Bio import Entrez, SeqIO
Entrez.email = 'nsrchemie@gmail.com'
hdl = Entrez.efetch(db='nucleotide', id=['NM_002299'], rettype='fasta')
seq = SeqIO.read(hdl, 'fasta')

In [15]:
from Bio import SeqIO
w_hdl = open('example.fasta', 'w')
w_seq = seq[11:5795]
SeqIO.write([w_seq], w_hdl, 'fasta')
w_hdl.close()

In [16]:
recs = SeqIO.parse('example.fasta', 'fasta')
for rec in recs:
    seq = rec.seq
    print(rec.description)
    print(seq[:10])
    print(seq.alphabet)

gi|974141076|ref|NM_002299.3| Homo sapiens lactase (LCT), mRNA
ATGGAGCTGT
SingleLetterAlphabet()


In [17]:
from Bio import Seq
from Bio.Alphabet import IUPAC
seq = Seq.Seq(str(seq), IUPAC.unambiguous_dna)

In [18]:
rna = Seq.Seq(str(seq), IUPAC.unambiguous_dna)
rna = seq.transcribe()
print(rna)


AUGGAGCUGUCUUGGCAUGUAGUCUUUAUUGCCCUGCUAAGUUUUUCAUGCUGGGGGUCAGACUGGGAGUCUGAUAGAAAUUUCAUUUCCACCGCUGGUCCUCUAACCAAUGACUUGCUGCACAACCUGAGUGGUCUCCUGGGAGACCAGAGUUCUAACUUUGUAGCAGGGGACAAAGACAUGUAUGUUUGUCACCAGCCACUGCCCACUUUCCUGCCAGAAUACUUCAGCAGUCUCCAUGCCAGUCAGAUCACCCAUUAUAAGGUAUUUCUGUCAUGGGCACAGCUCCUCCCAGCAGGAAGCACCCAGAAUCCAGACGAGAAAACAGUGCAGUGCUACCGGCGACUCCUCAAGGCCCUCAAGACUGCACGGCUUCAGCCCAUGGUCAUCCUGCACCACCAGACCCUCCCUGCCAGCACCCUCCGGAGAACCGAAGCCUUUGCUGACCUCUUCGCCGACUAUGCCACAUUCGCCUUCCACUCCUUCGGGGACCUAGUUGGGAUCUGGUUCACCUUCAGUGACUUGGAGGAAGUGAUCAAGGAGCUUCCCCACCAGGAAUCAAGAGCGUCACAACUCCAGACCCUCAGUGAUGCCCACAGAAAAGCCUAUGAGAUUUACCACGAAAGCUAUGCUUUUCAGGGCGGAAAACUCUCUGUUGUCCUGCGAGCUGAAGAUAUCCCGGAGCUCCUGCUAGAACCACCCAUAUCUGCGCUUGCCCAGGACACGGUCGAUUUCCUCUCUCUUGAUUUGUCUUAUGAAUGCCAAAAUGAGGCAAGUCUGCGGCAGAAGCUGAGUAAAUUGCAGACCAUUGAGCCAAAAGUGAAAGUUUUCAUCUUCAACCUAAAACUCCCAGACUGCCCCUCCACCAUGAAGAACCCAGCCAGUCUGCUCUUCAGCCUUUUUGAAGCCAUAAAUAAAGACCAAGUGCUCACCAUUGGGUUUGAUAUUAAUGAGUUUCUGAGUUGUUCAUCAAGUUCCAAGAAAAGCAUGUCUUGUUCUC

In [19]:
prot = seq.translate()
print(prot)

MELSWHVVFIALLSFSCWGSDWESDRNFISTAGPLTNDLLHNLSGLLGDQSSNFVAGDKDMYVCHQPLPTFLPEYFSSLHASQITHYKVFLSWAQLLPAGSTQNPDEKTVQCYRRLLKALKTARLQPMVILHHQTLPASTLRRTEAFADLFADYATFAFHSFGDLVGIWFTFSDLEEVIKELPHQESRASQLQTLSDAHRKAYEIYHESYAFQGGKLSVVLRAEDIPELLLEPPISALAQDTVDFLSLDLSYECQNEASLRQKLSKLQTIEPKVKVFIFNLKLPDCPSTMKNPASLLFSLFEAINKDQVLTIGFDINEFLSCSSSSKKSMSCSLTGSLALQPDQQQDHETTDSSPASAYQRIWEAFANQSRAERDAFLQDTFPEGFLWGASTGAFNVEGGWAEGGRGVSIWDPRRPLNTTEGQATLEVASDSYHKVASDVALLCGLRAQVYKFSISWSRIFPMGHGSSPSLPGVAYYNKLIDRLQDAGIEPMATLFHWDLPQALQDHGGWQNESVVDAFLDYAAFCFSTFGDRVKLWVTFHEPWVMSYAGYGTGQHPPGISDPGVASFKVAHLVLKAHARTWHHYNSHHRPQQQGHVGIVLNSDWAEPLSPERPEDLRASERFLHFMLGWFAHPVFVDGDYPATLRTQIQQMNRQCSHPVAQLPEFTEAEKQLLKGSADFLGLSHYTSRLISNAPQNTCIPSYDTIGGFSQHVNHVWPQTSSSWIRVVPWGIRRLLQFVSLEYTRGKVPIYLAGNGMPIGESENLFDDSLRVDYFNQYINEVLKAIKEDSVDVRSYIARSLIDGFEGPSGYSQRFGLHHVNFSDSSKSRTPRKSAYFFTSIIEKNGFLTKGAKRLLPPNTVNLPSKVRAFTFPSEVPSKAKVVWEKFSSQPKFERDLFYHGTFRDDFLWGVSSSAYQIEGAWDADGKGPSIWDNFTHTPGSNVKDNATGDIACDSYHQLDADLNMLRALKVKAYRFSISWSRIFPTGRNSSINSHGVDYY

In [23]:
import gzip
from Bio import SeqIO
recs = SeqIO.parse(gzip.open('SRR003265.filt.fastq.gz'), 'fastq')
rec = next(recs)
print(rec.id, rec.description, rec.seq)
print(rec.letter_annotations)

ValueError: Is this handle in binary mode not text mode?

In [26]:
from collections import defaultdict
recs = SeqIO.parse(gzip.open('SRR003265.filt.fastq (2).gz'), 'fastq')
cnt = defaultdict(int)
for rec in recs:
    for letter in rec.seq:
        cnt[letter] += 1
tot = sum(cnt.values())
for letter, cnt_value in cnt.items():
    print('%s: %.2f %d' % (letter, 100. * cnt_value / tot, cnt_value))

OSError: Not a gzipped file